In [ ]:
from main_con import con_matrix
import os
from scripts import func
from nilearn.image import mean_img
from nilearn import plotting
import glob as glob
from nilearn.plotting import plot_img, plot_epi, plot_roi, plot_stat_map
import importlib
importlib.reload(func)


In [ ]:
# --Data--

data_dir =r'E:\Users\Dylan\Desktop\UdeM_H22\E_PSY3008\data_desmartaux\2sub_ASL_data'
save_base = r'C:\Users\Dylan\Desktop\UM_Bsc_neurocog\E22\Projet_Ivado_rainvillelab\results'
atlas_name="difumo64"
atlas_type="maps"
data = func.load_data(data_dir)
conditions = ["pre_hyp", "post_hyp", "contrast"]
func_pre = data.func_pre_hyp
func_post = data.func_post_hyp
all_files = func_pre + func_post
results = dict(pre_series=list(), post_series=list())

atlas, atlas_labels,atlas_name, confounds = func.load_choose_atlas(atlas_name, bilat=True)



### Mean data masks on mean anat template

In [ ]:
from nilearn.image import  mean_img
mean_anat = mean_img(data.anat)
mean_data_mask = mean_img([data.pre_masks, data.post_masks])
plotting.plot_roi(mean_data_mask, bg_img=mean_anat, cmap='Paired')

# Data manip

In [ ]:
import nibabel as nib

pre_imgs = [nib.load(img) for img in pre_data]
resampled_series = pre_imgs



In [ ]:
from nilearn.image import concat_imgs, mean_img, resample_to_img
from nilearn.plotting import plot_epi, plot_roi, plot_stat_map
from nilearn.maskers import NiftiMasker
def check_masker_fit(data, masker):
    # print basic information on the dataset
    print("First functional nifti image (4D) is located " f"at: {data}")

    filename = data
    mean_im = mean_img(filename)
    plot_epi(mean_im, title="Mean EPI image")

    masker.fit(data)
    print("Masker fit done, see html report!")
    report = masker.generate_report()
    #report.save_as_html("masker_report.html")

    # plot the mask
    plot_roi(masker.mask_img_, mean_im, title="Mask")
masker = NiftiMasker(standardize=True, detrend=True, verbose=5)
check_masker_fit(data.post_hyp[1],masker)


### mean mask in MNI space

meanASL_4D_before : No
wmeanCBF_0_srASL_4D_before : yes

In [ ]:
import nibabel as nib

file = r'E:\Users\Dylan\Desktop\UdeM_H22\E_PSY3008\data_desmartaux\2sub_ASL_data\APM_08_H2\01-PCASL_before_hypnosis\wcbf_0_srASL_4D_before_4D.nii'
data_mask = nib.load(file)
plot_epi(data_mask, title="Mean EPI image")


In [ ]:
test_img = data.func_pre_hyp[1]
test_img = nib.load(test_img)
new_img = resample_to_img(index_img(test_img,0), data_mask, interpolation='nearest')
plot_roi(new_img, title = 'resampled to : wmeanCBF_0_srASL_4D_before.nii')

In [ ]:
file = r'E:\Users\Dylan\Desktop\UdeM_H22\E_PSY3008\data_desmartaux\2sub_ASL_data\APM_08_H2\01-PCASL_before_hypnosis\wcbf_0_srASL_4D_before_4D.nii'
mean_data_mask = nib.load(file)
plot_roi(mean_data_mask, title = 'mask')
plot_epi(mean_data_mask, title = 'mask')
print(data_mask.shape)

### 3D slice to show alignment and background


In [ ]:
from nilearn.image import resample_img, index_img
img = index_img(pre_data[0],1)
img = resample_img(img, target_affine =index_img(pre_data[0],1).affine , interpolation = 'nearest')
plot_roi(img, title = 'mask')
plot_epi(img, bg_img = mean_data_mask, title = 'mask')
print(img.shape)


## 1. checking affines from 4D nifti files

In [ ]:
import nibabel as nib
from nilearn.image import index_img
import numpy as np
#
#im = index_img(im, 0)
#print(im.shape, im.header.get_zooms(), im.get_fdata().max(), im.get_fdata().min(), im.affine)

def check_affines(im):
    compare = []
    for i in range(im.shape[-1]):
        aff = np.array(index_img(im, i).affine)
        compare.append(aff)
        if i > 0:
            if (aff == compare[i-1]).all() == False:
                print('Error, not same affines', i, aff, compare[i-1])
                break
def check_4D_affines(im):
    '''
    im : list of 4D images
    
    im is a list of 4D images
    check if all affines are the same
    for each subject
    '''
    compare = []
    for i in range(len(im)):
        if type(im[i]) == str:
            im[i] = nib.load(im[i])
            aff = np.array(nib.load(im[i]).affine)
        else:
            aff = np.array(im[i].affine)
        print(aff)
        compare.append(aff)
        if i > 0:
            if (aff == compare[i-1]).all() == False:
                print('Error, not same affines for all subjects', i, aff, compare[i-1])
                break 
    
def check_shape(images_list):
    compare = []
    for i in range(len(images_list)):
        sh = np.array(index_img(images_list, i).shape[-1])
        compare.append(sh)
        if i > 0:
            if (sh == compare[i-1]).all() == False:
                print('Error, not same shape for all subjects', i, sh, compare[i-1])
                break
    print('------------------')

check_shape(func_pre[1])
check_4D_affines(func_pre)




## 2. resampling affine and shapes

In [ ]:
from nilearn import plotting
from nilearn.image import mean_img

m_im = mean_img(func_pre)
print('mean affine', m_im.affine)
plot_roi(m_im, title = 'mask')
plotting.view_img(m_im, threshold=None)



 when target_affine = reference_affine, the shape of the resampled image is NOT the same as the reference image. Have to specify mask_strategy =  ref_img.shape[:-1]

In [ ]:
import nibabel as nib
mean_all = mean_img(func_post + func_pre)
print(mean_all.affine, mean_all.shape)
(mean_all.affine == nib.load(func_post[1]).affine).all()

#### Resampling to mean_img from all data

In [ ]:
from nilearn.image import resample_to_img
resampled = []
all_files = func_pre + func_post
ref_img = mean_all # with affine of first image used to fit mean_img
for i in range(len(all_files)):
    resampled.append(resample_to_img(all_files[i], ref_img, interpolation='continuous'))


In [ ]:
mean_resampled = mean_img(resampled)
mean_resampled.shape

#### Resampling to MNI space

In [ ]:
data_mask.shape

In [ ]:
mni_resampled = []
all_files = func_pre
ref_img = data_mask # with affine of first image used to fit mean_img
for i in range(len(all_files)):
    mni_resampled.append(resample_to_img(all_files[i], ref_img, interpolation='continuous'))
mean_mni = mean_img(mni_resampled)

### 2.1 Comparing resampled-to-img.affine VS resampled-to-mniMask

In [ ]:
mean_resampled = mean_img(resampled)    
plot_roi(mean_resampled, title = 'mean_resampled', colorbar=True)
plot_epi(mean_resampled, title = 'mean_resampled',colorbar=True)
plotting.view_img(mean_resampled, threshold=None)
plot_epi(mean_mni, title = 'mean resampled to mni mean mask',colorbar=True)
plot_roi(mean_mni, title = 'mean resampled to mni mean mask',colorbar=True)
plotting.view_img(mean_mni, threshold=None)

#### Resampling with masker

In [ ]:
from nilearn.maskers import NiftiMasker
from nilearn.image import resample_to_img
import nibabel as nib

ref_img = data_mask
all_files = pre_data[0]
resample_strategy = 'masker'
args = {}
resampled_series = None
if resample_strategy == 'masker':
    # using masker to resample all images to the same shape
    voxel_masker = NiftiMasker(target_affine= ref_img.affine,target_shape = ref_img.shape, t_r = 3, high_pass = 0.1, detrend = True,mask_strategy = 'background',mask_args = args, standardize='zscore_sample')
    voxel_masker.fit(all_files)
    masked_img = voxel_masker.transform(all_files)
    print('ref_img.shape :{}, masked image shape : {}, inverse transformed img shape {}'.format(ref_img.shape, masked_img.shape,voxel_masker.inverse_transform(masked_img).shape))  
    resampled_series = [voxel_masker.inverse_transform(im) for im in masked_img]

    print('ref_img.shape :{}, resampled imgages shapes {}'.format(ref_img.shape, [s.shape for s in resampled_series]))
    check_4D_affines(resampled_series)


### Computing an EPI mask

In [ ]:
from nilearn.maskers import NiftiMasker
from nilearn.plotting import plot_epi, plot_roi
from nilearn.masking import compute_epi_mask, compute_multi_epi_mask
multi_epi_mask = compute_multi_epi_mask(resampled_series, opening = 1, connected = True, exclude_zeros = True, 
                                       upper_cutoff = 0.95, lower_cutoff = 0.2)
plot_roi(multi_epi_mask, bg_img = m_im)

# fitting nifi masker on one subject to generate report
nm = NiftiMasker(mask_img = multi_epi_mask)
nm.fit(resampled_series[2])
nm.generate_report()

## 3. Maskers for resampled series

### NiftiMasker

In [ ]:
mean_resampled = mean_img(resampled)

Comparing mean resampled vs mean resampled to MNI

In [ ]:
from nilearn.image import concat_imgs
for img in [resampled, mni_resampled]:
    concat = mean_img(concat_imgs(img))
    print(concat.shape)
    plot_roi(concat, title = 'mask')
    v = plotting.view_img(concat, threshold=None)
    v

### A. Masker on resampled data, no mask_img on single-sub timeseries

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker
m = NiftiMasker(mask_strategy = 'whole-brain-template', t_r = 3, smoothing_fwhm=6).fit(resampled[0])
t = m.transform(resampled[0])
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(resampled[1]), title = 'mean data on mni background')
plot_roi(mean_img(trans_mean_mask), title = 'transfomed mean img')
plot_roi(m.mask_img_, title = 'computed mask')
m.generate_report()

In [ ]:
trans_mean_mask[0].affine == mni_resampled[0].affine

#### A.b with 'epi' strategy

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker
m = NiftiMasker(mask_strategy = 'epi', t_r = 3, smoothing_fwhm=6).fit(resampled[0])
t = m.transform(resampled[0])
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(trans_mean_mask), title = 'mean transformed imgs')
plot_roi(m.mask_img_, title = 'computed mask')
m.generate_report()

### B. Masker on MNI-resampled data, with mask_img on single-sub timeseries

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker

m = NiftiMasker(mask_strategy = 'whole-brain-template', t_r = 3, smoothing_fwhm=6).fit(mni_resampled[0])
t = m.transform(mni_resampled[0], confounds = data.confounds_pre_hyp + data.confounds_post_hyp)
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(trans_mean_mask), title = 'mean transformed imgs')
plot_roi(m.mask_img_, title = 'computed mask')
m.generate_report()

### C. Masker on MNI-resampled data, with binarized mean_data_mask(MNI) on single-sub timeseries

In [ ]:
from nilearn.image import binarize_img
bin_mni_mask = binarize_img(mean_data_mask)
plot_roi(bin_mni_mask, title = 'binarized mni mask')

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker

m = NiftiMasker(mask_img = bin_mni_mask, mask_strategy = 'whole-brain-template', t_r = 3, smoothing_fwhm=6).fit(mni_resampled[0])
t = m.transform(mni_resampled[0])
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(trans_mean_mask), title = 'mean transformed imgs')
plot_roi(m.mask_img_, title = 'computed mask')
m.generate_report()

### D. Masker on resampled data, with binarized mean_data_mask(MNI) on single-sub timeseries

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker

m = NiftiMasker(mask_img = bin_mni_mask, mask_strategy = 'whole-brain-template', t_r = 3, smoothing_fwhm=6).fit(resampled[0])
t = m.transform(resampled[0])
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(trans_mean_mask), title = 'mean transformed imgs')
plot_roi(m.mask_img_, title = 'computed mask')
m.generate_report()

### E. Comparing 'epi' vs 'background' strategies on  multisubjects resampled data

> 'epi' --> cuts a little the data, but removes out of brain ring 

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker

mnm = MultiNiftiMasker(mask_strategy = 'epi', t_r = 3, smoothing_fwhm=6).fit(resampled)
t = mnm.transform(resampled, confounds = data.confounds_pre_hyp + data.confounds_post_hyp)
trans_mean_mask = mnm.inverse_transform(t)
plot_roi(mean_img(resampled),bg_img = mean_all, title = 'mean imgs', colorbar=True)
plot_roi(mean_img(trans_mean_mask),bg_img = mean_all, title = 'mean transformed imgs', colorbar=True)
plot_roi(mnm.mask_img_,bg_img=mean_all, title = 'computed mask',  colorbar=True)


> 'background' --> keeps the data, but keeps out of brain ring

In [ ]:
from nilearn.maskers import MultiNiftiMasker, NiftiMasker

m = MultiNiftiMasker(mask_strategy = 'background',t_r = 3, smoothing_fwhm=6).fit(resampled)
t = m.transform(resampled, confounds = data.confounds_pre_hyp + data.confounds_post_hyp)
trans_mean_mask = m.inverse_transform(t)
plot_roi(mean_img(resampled),bg_img = mean_all, title = 'mean imgs', colorbar=True)
plot_roi(mean_img(trans_mean_mask),bg_img = mean_all, title = 'mean transformed imgs', colorbar=True)
plot_roi(m.mask_img_, title = 'computed mask',  colorbar=True)

#### Comparing masker.transform vs masker.transform_imgs : Appear to be the same at the 5D level!

In [ ]:
print(trans_[0].shape,transf_imgs[0].shape)
(trans_[0] == transf_imgs[0]).all()

#### Transforming single subject 4D data

## 4. Maps masker

In [ ]:
from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = NiftiMapsMasker(
            maps_img=atlas,
            mask_img = mnm.mask_img_,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
masker.fit(resampled[0])

time_series = masker.transform(resampled[0], confounds=data.confounds_pre_hyp[0])
plot_roi(
    masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker")

In [ ]:
from nilearn import plotting
from nilearn.image import index_img
from nilearn.plotting import find_xyz_cut_coords

# Showing region extraction results using 4D maps visualization tool
plotting.plot_prob_atlas(
    atlas,
    display_mode="z",
    cut_coords=1,
    view_type="contours",
    title="Regions extracted.",
)

In [ ]:
from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = NiftiMapsMasker(
            maps_img=atlas,
            mask_img = mean_data_mask,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="mask",
        )
masker.fit(all_files[0])

time_series = masker.transform(resampled[0], confounds=data.confounds_pre_hyp[0])
plot_roi(
    masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker")

In [ ]:
report = masker.generate_report(displayed_maps=[2, 6, 7, 16, 21, 30, 40,50,60])
report

In [ ]:
from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = NiftiMapsMasker(
            maps_img=atlas,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
masker.fit(resampled[0])

time_series = masker.transform(resampled[0], confounds=data.confounds_pre_hyp[0])
plot_roi(
    masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker")

In [ ]:
masker.generate_report()

## 5. labeled masker

### yeo17 atlas (labeled atlas vs maps)

In [ ]:
importlib.reload(func)

yeoatlas, atlas_labels,atlas_name, confounds = func.load_choose_atlas('yeo_17', bilat=True)


In [ ]:
plot_roi(mean_resampled, bg_img = resample_to_img(mean_resampled, yeoatlas))

In [ ]:
#check for matching labels/values
import numpy as np
print(atlas_labels, np.unique(yeoatlas.get_fdata(), return_counts=True))

In [ ]:
from nilearn.maskers import NiftiLabelsMasker, MultiNiftiLabelsMasker

lm = NiftiLabelsMasker(labels_img = yeoatlas, mask_img= bin_mni_mask, labels = atlas_labels,resampling_target = 'data', standardize = 'zscore_sample', verbose=0)
lm.fit(mni_resampled[1])
lm.generate_report()

In [ ]:
from nilearn import datasets

atlas = datasets.fetch_atlas_msdl()
# Loading atlas image stored in 'maps'
atlas_filename = atlas["maps"]
# Loading atlas data stored in 'labels'

from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = NiftiMapsMasker(
            maps_img=atlas_filename,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
masker.fit(resampled[0])

time_series = masker.transform(resampled[0], confounds=data.confounds_pre_hyp[0])
#plot_roi(
    #masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker")

In [ ]:
masker.generate_report(displayed_maps=[2, 6, 7,])


> MapsMasker with mask from MultiMasker ('epi' strategy)
--> not aligned with atlas!

In [ ]:
from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = MultiNiftiMapsMasker(
            maps_img=atlas,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
masker.fit(resampled)
results['pre_series'] = [masker.transform(ts, confounds = cf) for ts, cf in zip(resampled, data.confounds_pre_hyp)]
inv = [masker.inverse_transform(ts) for ts in results['pre_series']]
plot_roi(
    masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker"
)

In [ ]:
from nilearn.maskers import NiftiMapsMasker, MultiNiftiMapsMasker  

masker = MultiNiftiMapsMasker(
            maps_img=atlas,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
masker.fit(mni_resampled)
results['pre_series'] = [masker.transform(ts, confounds = cf) for ts, cf in zip(mni_resampled, data.confounds_pre_hyp)]
inv = [masker.inverse_transform(ts) for ts in results['pre_series']]
plot_roi(
    masker.mask_img_,bg_img = mean_img(atlas), title="Maps mask from multi-masker")

In [ ]:
atlas.shape

In [ ]:
results['pre_series'][0].shape
inv[0].shape

In [ ]:
from nilearn.maskers import MultiNiftiMapsMasker   

multi_mapsmasker = MultiNiftiMapsMasker(
            maps_img=atlas,
            mask_img=voxel_masker.mask_img_,
            t_r=3,
            smoothing_fwhm=6,
            standardize="zscore_sample",
            verbose=5,
            resampling_target="data",
        )
multi_mapsmasker.fit(resampled_series)
print(multi_mapsmasker.mask_img_.shape, multi_mapsmasker.transform(resampled_series[0]).shape)
samp_multimaps0 = multi_mapsmasker.transform(resampled_series[0])
plot_roi(
    masker.mask_img_,bg_img = m_im, title="Maps mask from multi-masker"
)

In [ ]:
multi_mapsmasker.generate_report()

#### Comparing single img transform from MultiMaps vs maps : gives same output 

In [ ]:
print(samp_maps0.shape, samp_multimaps0.shape)
(samp_maps0 == samp_multimaps0).all()

In [ ]:
from nilearn.plotting import plot_prob_atlas

plot_prob_atlas(atlas, bg_img = voxel_masker.mask_img_)

### Sphere-ROI correlation to plot

### test on all data

In [ ]:
print(results["pre_series"][0].shape, results["seed_pre_series"][0].shape)
[seed_masker.transform(ts) for ts in resampled_series]


In [ ]:
results["voxel_pre_series"] = [mnm.transform(ts) for ts in resampled_series]

In [ ]:
from nilearn import plotting
from nilearn.maskers import NiftiSpheresMasker

sphere_coord = [(54, -28, 26)]
seed_masker = NiftiSpheresMasker(
    sphere_coord, radius=8, standardize="zscore_sample"
)

seed_masker.fit(resampled_series)

results["seed_pre_series"] = [seed_masker.transform(ts) for ts in resampled_series]
results["seed_to_pre_correlations"] = [
        (np.dot(brain_time_series.T, seed_time_series) / seed_time_series.shape[0])
        for brain_time_series, seed_time_series in zip(
            results["voxel_pre_series"], results["seed_pre_series"]
        )
    ]    

results["mean_seed_pre_connectome"] = np.mean(
        results["seed_to_pre_correlations"], axis=0
    )

    

In [ ]:
results["mean_seed_pre_connectome"].T.shape


In [ ]:
from nilearn.datasets import fetch_atlas_yeo_2011

# Fetch the Yeo-17 atlas
atlas = fetch_atlas_yeo_2011()

# Access the region labels
region_labels = atlas['labels']

# Print the region labels
for label in region_labels:
    print(label)

In [ ]:

seed_to_voxel_correlations_img = mnm.inverse_transform(
    results["mean_seed_pre_connectome"].T
)

display = plotting.plot_stat_map(
    seed_to_voxel_correlations_img,
    threshold=0.5,
    vmax=1,
    cut_coords=sphere_coord[0],
    title="Seed-to-voxel correlation (OP seed)",
)
display.add_markers(
    marker_coords=sphere_coord[0], marker_color="g", marker_size=300
)

In [ ]:
from nilearn.datasets import fetch_miyawaki2008

dataset = fetch_miyawaki2008()

# training data starts after the first 12 files
fmri_random_runs_filenames = dataset.func[12:]
stimuli_random_runs_filenames = dataset.label[12:]

# training data starts after the first 12 files
fmri_random_runs_filenames = dataset.func[12:]
stimuli_random_runs_filenames = dataset.label[12:]

import numpy as np

from nilearn.maskers import MultiNiftiMasker

masker = MultiNiftiMasker(
    mask_img=dataset.mask, detrend=True, standardize="zscore_sample"
)
masker.fit()
fmri_data = masker.transform(fmri_random_runs_filenames)

# shape of the binary (i.e. black and wihte values) image in pixels
stimulus_shape = (10, 10)

# We load the visual stimuli from csv files
stimuli = []
for stimulus_run in stimuli_random_runs_filenames:
    stimuli.append(
        np.reshape(
            np.loadtxt(stimulus_run, dtype=int, delimiter=","),
            (-1,) + stimulus_shape,
            order="F",
        )
    )
    